In [1]:
pip install pyspark

  Using cached https://files.pythonhosted.org/packages/86/ec/60880978512d5569ca4bf32b3b4d7776a528ecf4bca4523936c98c92a3c8/py4j-0.10.9.5-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import sys
os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark

In [4]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [13]:
# Path to the CSV file
#file_path = '/kaggle/input/nasa-nearest-earth-objects-1910-2024/nearest-earth-objects(1910-2024).csv'


file_path = "file:///home/talentum/shared/NEO_Project/nearest-earth-objects(1910-2024).csv"


# Read the CSV file into a DataFrame
neo = spark.read.csv(file_path, header=True, inferSchema=True)
# Show the first few rows of the DataFrame
neo.show(5)


+-------+------------------+------------------+----------------------+----------------------+-------------+-----------------+--------------------+------------+
| neo_id|              name|absolute_magnitude|estimated_diameter_min|estimated_diameter_max|orbiting_body|relative_velocity|       miss_distance|is_hazardous|
+-------+------------------+------------------+----------------------+----------------------+-------------+-----------------+--------------------+------------+
|2162117|162117 (1998 SD15)|             19.14|          0.3949616937|          0.8831611957|        Earth| 71745.4010476829| 5.814362331916972E7|       false|
|2349507|  349507 (2008 QY)|              18.5|          0.5303407233|          1.1858779086|        Earth|109949.7571484926| 5.580104781819944E7|        true|
|2455415|  455415 (2003 GA)|             21.45|          0.1363185559|          0.3048175575|        Earth| 24865.5067981164| 6.720688772254463E7|       false|
|3132126|         (2002 PB)|            

In [14]:
neo.printSchema()
#printschema shows all columns

root
 |-- neo_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- absolute_magnitude: double (nullable = true)
 |-- estimated_diameter_min: double (nullable = true)
 |-- estimated_diameter_max: double (nullable = true)
 |-- orbiting_body: string (nullable = true)
 |-- relative_velocity: double (nullable = true)
 |-- miss_distance: double (nullable = true)
 |-- is_hazardous: boolean (nullable = true)



In [15]:
neo.show()

+--------+-------------------+------------------+----------------------+----------------------+-------------+-----------------+--------------------+------------+
|  neo_id|               name|absolute_magnitude|estimated_diameter_min|estimated_diameter_max|orbiting_body|relative_velocity|       miss_distance|is_hazardous|
+--------+-------------------+------------------+----------------------+----------------------+-------------+-----------------+--------------------+------------+
| 2162117| 162117 (1998 SD15)|             19.14|          0.3949616937|          0.8831611957|        Earth| 71745.4010476829| 5.814362331916972E7|       false|
| 2349507|   349507 (2008 QY)|              18.5|          0.5303407233|          1.1858779086|        Earth|109949.7571484926| 5.580104781819944E7|        true|
| 2455415|   455415 (2003 GA)|             21.45|          0.1363185559|          0.3048175575|        Earth| 24865.5067981164| 6.720688772254463E7|       false|
| 3132126|          (2002 PB

In [17]:
neo.describe().show()

+-------+--------------------+--------------------+------------------+----------------------+----------------------+-------------+------------------+--------------------+
|summary|              neo_id|                name|absolute_magnitude|estimated_diameter_min|estimated_diameter_max|orbiting_body| relative_velocity|       miss_distance|
+-------+--------------------+--------------------+------------------+----------------------+----------------------+-------------+------------------+--------------------+
|  count|              338199|              338199|            338171|                338171|                338171|       338199|            338199|              338199|
|   mean|1.7599388682018574E7|                null| 22.93252495926504|   0.15781204666053292|    0.3528784640005324|         null| 51060.66290759541|4.1535350932190806E7|
| stddev|2.2872246428931765E7|                null|2.9112163902923123|    0.3138851378796799|    0.7018685054244403|         null|26399.238434899

In [18]:
neo.count()

338199

In [23]:
neo.select("neo_id").distinct().show()

+--------+
|  neo_id|
+--------+
|54342500|
| 3753855|
| 3843134|
| 2523781|
|54131590|
| 3608092|
|54123253|
| 2286080|
|54202331|
| 3785785|
| 3545630|
|54117567|
|54435206|
| 3781455|
| 3648469|
|54122968|
| 3581506|
| 3076550|
| 3601665|
| 2086067|
+--------+
only showing top 20 rows



In [21]:
neo.select("name").distinct().count()

33514

In [30]:
from pyspark.sql.functions import col, isnan, when, count

In [34]:
null_counts = neo.select([count(when(col("absolute_magnitude").isNull() | isnan("absolute_magnitude"), absolute_magnitude)).alias("absolute_magnitude") for absolute_magnitude in neo.columns])

null_counts.show()

+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|absolute_magnitude|absolute_magnitude|absolute_magnitude|absolute_magnitude|absolute_magnitude|absolute_magnitude|absolute_magnitude|absolute_magnitude|absolute_magnitude|
+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|                28|                28|                28|                28|                28|                28|                28|                28|                28|
+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+



In [46]:
from pyspark.sql.types import DoubleType, StructField, StructType

In [48]:
magnitude_mean_values = neo.select([mean("absolute_magnitude").alias("absolute_magnitude") for absolute_magnitude in neo.columns if neo.schema["absolute_magnitude"].dataType == DoubleType()])
magnitude_mean_values.show()

NameError: name 'mean' is not defined